In [1]:
import os
import sys

os.chdir("..")
sys.path.append("..")

In [2]:
import itertools
import yaml

import numpy as np
import pandas as pd
import torch

from bokeh.io.export import export_svg
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.transform import log_cmap, linear_cmap
from bokeh.util.hex import hexbin, cartesian_to_axial
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.loader import ValidationDataLoader
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.time_feature import (
    HourOfDay,
    DayOfWeek,
    DayOfMonth,
    DayOfYear,
    MonthOfYear
)
from gluonts.torch.batchify import batchify
from gluonts.transform import (
    AddObservedValuesIndicator,
    AddTimeFeatures,
    Chain,
    InstanceSplitter,
    ValidationSplitSampler
)
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.seasonal import STL
from tqdm import tqdm

from src.models.utils import get_model
from src.utils.data_loading import load_features, load_score, load_test_data
from src.utils.evaluation import score_batch
from src.utils.features import decomps_and_features
from src.utils.transformations import manipulate_trend_component, manipulate_seasonal_determination

/anaconda/envs/whatif/lib/python3.8/site-packages/gluonts/json.py:45: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [3]:
def load_generated_data(prefix, generated_test_datadir, len_test_data):
    data = [0 for _ in range(len_test_data)]
    for f in os.listdir(generated_test_datadir):
        if f.startswith(prefix):
            file_name = f.split(".")[0]  # slice off .npy from file name
            ts_idx = int(file_name[len(prefix):])  # the remaining charachters after prefix is always the time series id
            data[ts_idx] = np.load(os.path.join(generated_test_datadir, f))

    data = np.array(data)  # [len(original_test), num_manipulations, 4]
    return data


def load_generated_features_and_data(suffix, suffix_prefix_to_fname, suffix_suffix_to_index, original_config, generated_datadir, len_test_data):
    f_suffix = get_file_suffix(suffix, suffix_prefix_to_fname)
    data_prefix = f"ts_{f_suffix}"
    feature_prefix = f"feat_{f_suffix}"
    gen_ts = load_generated_data(data_prefix, generated_datadir, len_test_data)
    gen_features = load_generated_data(feature_prefix, generated_datadir, len_test_data)

    idx = suffix_suffix_to_index[suffix[-3:]]
    gen_ts = gen_ts[:, idx, :].reshape([-1, original_config["context_length"] + original_config["prediction_length"]])
    gen_features = gen_features[:, idx, :].reshape([-1, 4])  # flatten the two first dimensions
    return gen_ts, gen_features


def create_gen_dataloader(generated_data, dataset, context_length, prediction_length, batch_size):
    original_dataset = get_dataset(dataset)
    
    # nbeats doesn't do any covariates so we don't care that the starting points of forecasts are wrong here
    list_data = [{"start": original["start"], "target": generated, "feat_static_cat": original["feat_static_cat"], "item_id": original["item_id"]}
                 for original, generated in zip(itertools.cycle(original_dataset.test), generated_data)]
    generated_dataset = ListDataset(list_data, freq=original_dataset.metadata.freq)
    
    transformation = Chain([
        AddObservedValuesIndicator(
            target_field=FieldName.TARGET,
            output_field=FieldName.OBSERVED_VALUES,
        ),
        AddTimeFeatures(
            start_field=FieldName.START,
            target_field=FieldName.TARGET,
            output_field=FieldName.FEAT_TIME,
            pred_length=prediction_length,
            time_features=[HourOfDay(), DayOfWeek(), DayOfMonth(), DayOfYear(), MonthOfYear()]
        ),
        InstanceSplitter(
            target_field=FieldName.TARGET,
            is_pad_field=FieldName.IS_PAD,
            start_field=FieldName.START,
            forecast_start_field=FieldName.FORECAST_START,
            instance_sampler=ValidationSplitSampler(min_future=prediction_length),
            past_length=context_length,
            future_length=prediction_length,
            time_series_fields=[FieldName.FEAT_TIME, FieldName.OBSERVED_VALUES]
        )
    ])
    dataloader = ValidationDataLoader(
        generated_dataset,
        batch_size=batch_size,
        stack_fn=batchify,
        transform=transformation,
        num_workers=1
    )
    return dataloader


def organize_scores(all_scores, metric):
    mape = []
    smape = []
    mase = []
    seasonal_mase = []
    mse = []
    mae = []
    for score in all_scores:
        mape.append(score[0])
        smape.append(score[1])
        mase.append(score[2])
        seasonal_mase.append(score[3])
        mse.append(score[4])
        mae.append(score[5])


    mape = np.vstack(mape)
    smape = np.vstack(smape)
    mase = np.vstack(mase)
    seasonal_mase = np.vstack(seasonal_mase)
    mse = np.vstack(mse)
    mae = np.vstack(mae)
    
    return {"mape": mape, "smape": smape, "mase": mase, "seasonal_mase": seasonal_mase, "mse": mse, "mae": mae}[metric]


def get_scores(original_model, new_model, original_config, new_config, gen_ts, dataset, metric):

    gen_dataloader = create_gen_dataloader(gen_ts, dataset, original_config["trainer_args"]["context_length"],
                                           original_config["trainer_args"]["prediction_length"], original_config["trainer_args"]["batch_size"])

    original_gen_scores = []
    new_gen_scores = []
    original_model.eval()
    new_model.eval()
    for batch in tqdm(gen_dataloader):
        original_preds = original_model.predict(batch)[:, :, 0]
        new_preds = new_model.predict(batch)[:, :, 0]

        context = batch["past_target"].unsqueeze(dim=-1).numpy()
        target = batch["future_target"].numpy()

        original_gen_scores.append(score_batch(target, original_preds, context, original_config["sp"]))
        new_gen_scores.append(score_batch(target, new_preds, context, new_config["sp"]))

    original_gen_scores = organize_scores(original_gen_scores, metric)
    new_gen_scores = organize_scores(new_gen_scores, metric)
    
    return original_gen_scores, new_gen_scores

In [4]:
def create_instance_space(datadir):
    train_features = load_features(datadir, train=True)
    test_features = load_features(datadir, train=False)
    scaler = StandardScaler()
    norm_train_features = scaler.fit_transform(train_features)
    norm_test_features = scaler.transform(test_features)

    pca = PCA(n_components=2)
    train_pca_data = pca.fit_transform(norm_train_features)
    test_pca_data = pca.transform(norm_test_features)
    return pca, scaler, test_pca_data

In [5]:
def create_color_arr(scores, quantiles=None):
    if quantiles is None:
        quantiles = np.nanquantile(scores, [0.25, 0.75])
    
    low = scores < quantiles[0]
    high = scores > quantiles[1]
    medium = np.logical_and(~low, ~high)
    
    colors = np.empty_like(scores)
    colors[low] = 0
    colors[medium] = 1
    colors[high] = 2
    
    return colors


def color_bin(bins, pca_data, colors, column_name):
    q, r = cartesian_to_axial(pca_data[:, 0], pca_data[:, 1], 0.1, "pointytop")
    df = pd.DataFrame(dict(r=r, q=q))
    groups = df.groupby(["q", "r"])
    
    for (q, r), indexes in groups.groups.items():
        color = np.nanmean(colors[indexes])
        bins.loc[(bins["q"] == q) & (bins["r"] == r), column_name] = color
    
    return bins


def get_fig(title, xrange, yrange):
    p = figure(title=title, tools="", match_aspect=True, x_range=xrange, y_range=yrange)
    p.output_backend = "svg"
    p.title.align = "center"
    p.grid.visible = False
    return p


def create_and_plot_hexbin(original_scores, new_scores, pca_data, figdir, dataset, suffix, model, metric, limits):
    # create a seperate folder for each dataset and model
    figdir = os.path.join(figdir, dataset, model)
    if not os.path.isdir(figdir):
        os.makedirs(figdir, exist_ok=True)
    
    # create hexbins
    bins = hexbin(pca_data[:, 0], pca_data[:, 1], 0.1)
    bins["original_colors"] = np.nan
    bins["new_colors"] = np.nan
    
    orig_quantiles = np.nanquantile(original_scores, [0.25, 0.75])
    original_colors = create_color_arr(original_scores, orig_quantiles)
    new_colors = create_color_arr(new_scores, orig_quantiles)

    bins = color_bin(bins, pca_data, original_colors, "original_colors")
    bins = color_bin(bins, pca_data, new_colors, "new_colors")
    
    # plot original model
    if limits is not None:
        xrange = limits["xrange"]
        yrange = limits["yrange"]
    else:
        xrange = None
        yrange = None
    
    p = get_fig(f"{model} trained with original training data", xrange, yrange)
    p.hex_tile(q="q", r="r", size=0.1, line_color=None, source=bins,
                fill_color=linear_cmap("original_colors", "Viridis256", min(bins.original_colors), max(bins.original_colors)))

    export_svg(p, filename=os.path.join(figdir, f"{dataset}_{suffix}_{model}_orig_{metric}_hexbin.svg"))
    
    # plot new model
    p = get_fig(f"{model} trained with augmented training data", xrange, yrange)
    p.hex_tile(q="q", r="r", size=0.1, line_color=None, source=bins,
                fill_color=linear_cmap("new_colors", "Viridis256", min(bins.new_colors), max(bins.new_colors)))

    export_svg(p, filename=os.path.join(figdir, f"{dataset}_{suffix}_{model}_OOD_{metric}_hexbin.svg"))

In [6]:
def get_file_suffix(suffix, suffix_prefix_to_fname):
    for prefix in suffix_prefix_to_fname.keys():
        if suffix.startswith(prefix):
            return suffix_prefix_to_fname[prefix]

In [7]:
def generate_data(dataset, config):
    test_data = load_test_data(dataset, config["context_length"] + config["prediction_length"])
    
    trend_str_inc_ts = []
    trend_str_dec_ts = []
    trend_lin_inc_ts = []
    trend_lin_dec_ts = []
    trend_slope_inc_ts = []
    trend_slope_dec_ts = []
    seas_str_inc_ts = []
    seas_str_dec_ts = []

    trend_str_inc_feat = []
    trend_str_dec_feat = []
    trend_lin_inc_feat = []
    trend_lin_dec_feat = []
    trend_slope_inc_feat = []
    trend_slope_dec_feat = []
    seas_str_inc_feat = []
    seas_str_dec_feat = []
    for ts in tqdm(test_data):
        decomp = decomps_and_features([ts], config["sp"])[0][0]
        
        inc_str = manipulate_trend_component(decomp.trend, f=100, g=1, h=1, m=0) + decomp.seasonal + decomp.resid
        dec_str = manipulate_trend_component(decomp.trend, f=0.01, g=1, h=1, m=0) + decomp.seasonal + decomp.resid
        
        inc_lin = manipulate_trend_component(decomp.trend, f=1, g=1, h=100, m=0) + decomp.seasonal + decomp.resid
        dec_lin = manipulate_trend_component(decomp.trend, f=1, g=1, h=0.01, m=0) + decomp.seasonal + decomp.resid
        
        inc_slope = manipulate_trend_component(decomp.trend, f=1, g=1, h=1, m=-1) + decomp.seasonal + decomp.resid
        dec_slope = manipulate_trend_component(decomp.trend, f=1, g=1, h=1, m=1) + decomp.seasonal + decomp.resid
        
        generated_ts = [inc_str, dec_str, inc_lin, dec_lin, inc_slope, dec_slope]
        if config["sp"] > 1:
            inc_seas = manipulate_seasonal_determination(decomp.seasonal, k=100) + decomp.trend + decomp.resid
            dec_seas = manipulate_seasonal_determination(decomp.seasonal, k=0.01) + decomp.trend + decomp.resid
            generated_ts.extend([inc_seas, dec_seas])
            
        _, features = decomps_and_features(generated_ts, config["sp"])
        
        trend_str_inc_ts.append(inc_str)
        trend_str_dec_ts.append(dec_str)
        trend_lin_inc_ts.append(inc_lin)
        trend_lin_dec_ts.append(dec_lin)
        trend_slope_inc_ts.append(inc_slope)
        trend_slope_dec_ts.append(dec_slope)
        
        trend_str_inc_feat.append(features[0])
        trend_str_dec_feat.append(features[1])
        trend_lin_inc_feat.append(features[2])
        trend_lin_dec_feat.append(features[3])
        trend_slope_inc_feat.append(features[4])
        trend_slope_dec_feat.append(features[5])
        
        if config["sp"] > 1:
            seas_str_inc_ts.append(inc_seas)
            seas_str_dec_ts.append(dec_seas)
            seas_str_inc_feat.append(features[6])
            seas_str_dec_feat.append(features[7])
    
    trend_str_inc_ts = np.array(trend_str_inc_ts)
    trend_str_dec_ts = np.array(trend_str_dec_ts)
    trend_lin_inc_ts = np.array(trend_lin_inc_ts)
    trend_lin_dec_ts = np.array(trend_lin_dec_ts)
    trend_slope_inc_ts = np.array(trend_slope_inc_ts)
    trend_slope_dec_ts = np.array(trend_slope_dec_ts)
    seas_str_inc_ts = np.array(seas_str_inc_ts)
    seas_str_dec_ts = np.array(seas_str_dec_ts)
    
    trend_str_inc_feat = np.array(trend_str_inc_feat)
    trend_str_dec_feat = np.array(trend_str_dec_feat)
    trend_lin_inc_feat = np.array(trend_lin_inc_feat)
    trend_lin_dec_feat = np.array(trend_lin_dec_feat)
    trend_slope_inc_feat = np.array(trend_slope_inc_feat)
    trend_slope_dec_feat = np.array(trend_slope_dec_feat)
    seas_str_inc_feat = np.array(seas_str_inc_feat)
    seas_str_dec_feat = np.array(seas_str_dec_feat)
    
    ts_dict = {"trend_str_inc": trend_str_inc_ts, "trend_str_dec": trend_str_dec_ts,
               "lin_inc": trend_lin_inc_ts, "lin_dec": trend_lin_dec_ts,
               "slope_inc": trend_slope_inc_ts, "slope_dec": trend_slope_dec_ts,
               "seas_inc": seas_str_inc_ts, "seas_dec": seas_str_dec_ts}
    
    feat_dict = {"trend_str_inc": trend_str_inc_feat, "trend_str_dec": trend_str_dec_feat,
                "lin_inc": trend_lin_inc_feat, "lin_dec": trend_lin_dec_feat,
                "slope_inc": trend_slope_inc_feat, "slope_dec": trend_slope_dec_feat,
                "seas_inc": seas_str_inc_feat, "seas_dec": seas_str_dec_feat}
    
    return ts_dict, feat_dict

In [8]:
metric = "mae"
create_plots = True
models = ["feedforward", "seq2seq", "nbeats_g", "tcn", "transformer"]
model_names = ["Fully-connected", "LSTM", "N-BEATS", "TCN", "Transformer"]

dataset_suffixes = {
    "electricity_nips": ["seas_dec", "slope_dec", "slope_inc"],
    "traffic_nips": ["seas_dec", "slope_dec", "slope_inc", "trend_str_dec"],
    "m4_hourly": ["seas_dec", "slope_dec", "slope_inc"],
    "m4_daily": ["seas_inc"],
    "m4_weekly": ["slope_dec", "slope_inc", "trend_str_dec"],
    "m4_monthly": ["seas_dec", "slope_dec", "slope_inc", "lin_dec", "trend_str_dec"],
    "m4_quarterly": ["seas_dec", "seas_inc", "lin_dec", "slope_dec"],
    "m4_yearly": ["lin_dec", "slope_dec", "slope_inc"],
}
plot_limits = {
    "electricity_nips": {"slope_dec": {"xrange": [-5, 6], "yrange": [-8, 6]}, "slope_inc": {"xrange": [-3, 8], "yrange": [-3, 8]}},
    "traffic_nips": {"lin_dec": {"xrange": [-4, 5], "yrange": [-4, 5]}},
    "m4_hourly": {"slope_dec": {"xrange": [-10, 5], "yrange": [-12, 6]}},
    "m4_daily": {},
    "m4_weekly": {},
    "m4_monthly": {"lin_dec": {"xrange": [-5, 5], "yrange": [-5, 5]}, "slope_dec": {"xrange": [-5, 8], "yrange": [-5, 8]}, "trend_str_dec": {"xrange": [-5, 5], "yrange": [-5, 8]}},
    "m4_quarterly": {"lin_dec": {"xrange": [-4, 5], "yrange": [-4, 5]}, "slope_dec": {"xrange": [-4, 5], "yrange": [-4, 5]}},
    "m4_yearly": {"slope_dec": {"xrange": [-10, 10], "yrange": [-15, 5]}, "slope_inc": {"xrange": [-10, 10], "yrange": [-5, 15]}, "lin_dec": {"xrange": [-5, 5], "yrange": [-5, 5]}},
}
suffix_prefix_to_fname = {"seas": "seasonal_str", "slope": "trend_slope", "lin": "trend_lin", "trend_str": "trend_str"}
suffix_suffix_to_index = {"inc": 98, "dec": -1}

figdir = "figures/OOD"
if not os.path.isdir(figdir):
    os.makedirs(figdir, exist_ok=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
scores_dict = {key: {} for key in dataset_suffixes.keys()}

for dataset in dataset_suffixes.keys():
    print(f"Calculating scores and creating plots for {dataset}...")
    datadir = f"data/{dataset}"
    generated_datadir = os.path.join(f"/datadrive2/whatif/{dataset}", "generated", "test")
    
    #create instance space
    pca, scaler, test_pca_data = create_instance_space(datadir)
    
    # load the config and score of some random model to get metadata
    original_experiment_dir = f"experiments/{dataset}/nbeats_g"
    with open(os.path.join(original_experiment_dir, "config.yaml"), "r") as f:
        original_config = yaml.load(f, Loader=yaml.FullLoader)
        
    original_scores = load_score(original_experiment_dir, metric)
    
    if dataset in ["m4_daily", "m4_weekly", "m4_yearly"]:
        generated_data, generated_features = generate_data(dataset, original_config)
    
    for suffix in dataset_suffixes[dataset]:
        # load data and features
        if suffix == "all":
            gen_ts = []
            gen_features = []
            for suffix in dataset_suffixes[dataset]:
                if suffix == "all":
                    continue
                
                if dataset in ["m4_daily", "m4_weekly", "m4_yearly"]:
                    gen_ts_suffix = generated_data[suffix]
                    gen_features_suffix = generated_features[suffix]
                else:
                    gen_ts_suffix, gen_features_suffix = load_generated_features_and_data(suffix, suffix_prefix_to_fname, suffix_suffix_to_index,
                                                                                          original_config, generated_datadir, original_scores.shape[0])
                gen_ts.append(gen_ts_suffix)
                gen_features.append(gen_features_suffix)

            gen_ts = np.vstack(gen_ts)
            gen_features = np.vstack(gen_features)
        else:
            if dataset in ["m4_daily", "m4_weekly", "m4_yearly"]:
                gen_ts = generated_data[suffix]
                gen_features = generated_features[suffix]
            else:
                gen_ts, gen_features = load_generated_features_and_data(suffix, suffix_prefix_to_fname, suffix_suffix_to_index,
                                                                        original_config, generated_datadir, original_scores.shape[0])

        for model, name in zip(models, model_names):
            # load original model
            original_experiment_dir = f"experiments/{dataset}/{model}"
            with open(os.path.join(original_experiment_dir, "config.yaml"), "r") as f:
                original_config = yaml.load(f, Loader=yaml.FullLoader)

            original_model = get_model(original_config["model_name"])(**original_config["model_args"], device=device, path=original_config["path"]).to(device)
            original_model.load_state_dict(torch.load(os.path.join(original_config["path"], "model.pth")))
            original_scores = load_score(original_experiment_dir, metric)
        
            # load new model
            new_experiment_dir = f"experiments/{dataset}/{model}_gen_{suffix}"
            with open(os.path.join(new_experiment_dir, "config.yaml"), "r") as f:
                new_config = yaml.load(f, Loader=yaml.FullLoader)
                
            new_model = get_model(new_config["model_name"])(**new_config["model_args"], device=device, path=new_config["path"]).to(device)
            new_model.load_state_dict(torch.load(os.path.join(new_config["path"], "model.pth")))
            new_scores = load_score(new_experiment_dir, metric)
            
            # evaluate models on OOD test data
            original_gen_scores, new_gen_scores = get_scores(original_model, new_model, original_config, new_config, gen_ts, dataset, metric)
            
            if name not in scores_dict[dataset].keys():
                scores_dict[dataset][name] = {suffix: {"original model": {"orig": np.nanmean(original_scores), "ood": np.nanmean(original_gen_scores)},
                                                       "new model": {"orig": np.nanmean(new_scores), "ood": np.nanmean(new_gen_scores)}}}
            else:
                scores_dict[dataset][name][suffix] = {"original model": {"orig": np.nanmean(original_scores), "ood": np.nanmean(original_gen_scores)},
                                                      "new model": {"orig": np.nanmean(new_scores), "ood": np.nanmean(new_gen_scores)}}
            
            # concatenate scores on original test data and ood test, and calculate mean per time series
            original_scores_concat = np.concatenate([original_scores, original_gen_scores], axis=0)
            new_scores_concat = np.concatenate([new_scores, new_gen_scores], axis=0)
            original_ts_scores = np.nanmean(original_scores_concat, axis=-1)
            new_ts_scores = np.nanmean(new_scores_concat, axis=-1)
            
            # transform generated features to instance space and concatenate with original test data
            norm_gen_features = scaler.transform(gen_features)
            gen_pca_data = pca.transform(norm_gen_features)
            concatenated_pca_data = np.concatenate([test_pca_data, gen_pca_data], axis=0)
            
            if create_plots:
                limits = plot_limits[dataset].get(suffix)
                create_and_plot_hexbin(original_ts_scores, new_ts_scores, concatenated_pca_data, figdir, dataset, suffix, name, metric, limits)

Calculating scores and creating plots for electricity_nips...


6it [00:01,  3.82it/s]
6it [00:02,  2.88it/s]
6it [00:02,  2.99it/s]
6it [00:03,  1.60it/s]
6it [00:03,  2.00it/s]
6it [00:01,  4.18it/s]
6it [00:02,  2.90it/s]
6it [00:01,  3.11it/s]
6it [00:03,  1.56it/s]
6it [00:03,  1.96it/s]
6it [00:01,  4.21it/s]
6it [00:02,  2.98it/s]
6it [00:01,  3.10it/s]
6it [00:03,  1.57it/s]
6it [00:03,  1.96it/s]


Calculating scores and creating plots for traffic_nips...


14it [00:03,  3.80it/s]
14it [00:05,  2.59it/s]
14it [00:04,  2.83it/s]
14it [00:09,  1.46it/s]
14it [00:07,  1.76it/s]
14it [00:03,  3.76it/s]
14it [00:05,  2.62it/s]
14it [00:05,  2.77it/s]
14it [00:09,  1.47it/s]
14it [00:08,  1.74it/s]
14it [00:03,  3.77it/s]
14it [00:05,  2.73it/s]
14it [00:04,  2.86it/s]
14it [00:09,  1.47it/s]
14it [00:07,  1.77it/s]
14it [00:03,  3.74it/s]
14it [00:05,  2.58it/s]
14it [00:05,  2.77it/s]
14it [00:09,  1.45it/s]
14it [00:07,  1.78it/s]


Calculating scores and creating plots for m4_hourly...


...

94it [00:12,  7.45it/s]
94it [00:17,  5.45it/s]
94it [00:21,  4.42it/s]
94it [00:23,  4.08it/s]
94it [00:22,  4.12it/s]
94it [00:12,  7.34it/s]
94it [00:17,  5.45it/s]
94it [00:21,  4.45it/s]
94it [00:22,  4.21it/s]
94it [00:22,  4.18it/s]
94it [00:12,  7.38it/s]
94it [00:17,  5.39it/s]
94it [00:20,  4.48it/s]
94it [00:22,  4.21it/s]
94it [00:22,  4.15it/s]
94it [00:12,  7.34it/s]
94it [00:16,  5.56it/s]
94it [00:20,  4.51it/s]
94it [00:22,  4.24it/s]
94it [00:22,  4.19it/s]
94it [00:12,  7.42it/s]
94it [00:17,  5.45it/s]
94it [00:20,  4.50it/s]
94it [00:22,  4.19it/s]
94it [00:22,  4.18it/s]


Calculating scores and creating plots for m4_quarterly...


47it [00:04,  9.89it/s]
47it [00:05,  7.87it/s]
47it [00:08,  5.33it/s]
47it [00:06,  7.19it/s]
47it [00:07,  6.43it/s]
47it [00:04,  9.92it/s]
47it [00:05,  7.94it/s]
47it [00:08,  5.30it/s]
47it [00:06,  7.28it/s]
47it [00:07,  6.40it/s]
47it [00:04,  9.75it/s]
47it [00:05,  8.11it/s]
47it [00:08,  5.24it/s]
47it [00:06,  7.29it/s]
47it [00:07,  6.39it/s]
47it [00:04,  9.65it/s]
47it [00:05,  8.20it/s]
47it [00:08,  5.38it/s]
47it [00:06,  7.50it/s]
47it [00:07,  6.48it/s]
  1%|          | 238/23000 [00:00<00:09, 2372.42it/s]

Calculating scores and creating plots for m4_yearly...
Loading test data


100%|██████████| 23000/23000 [12:02<00:00, 31.83it/s]
45it [00:05,  8.72it/s]
45it [00:05,  7.58it/s]
45it [00:09,  5.00it/s]
45it [00:06,  7.20it/s]
45it [00:07,  6.29it/s]
45it [00:05,  8.79it/s]
45it [00:05,  7.53it/s]
45it [00:08,  5.06it/s]
45it [00:06,  7.24it/s]
45it [00:07,  6.39it/s]
45it [00:05,  8.61it/s]
45it [00:05,  7.70it/s]
45it [00:08,  5.01it/s]
45it [00:06,  7.23it/s]
45it [00:07,  6.30it/s]


In [10]:
suffix_to_transformation = {"trend_str_inc": " (f=100)", "trend_str_dec": " (f=0.01)",
                            "slope_inc": " (m=1)", "slope_dec": " (m=-1)",
                            "lin_inc": " (h=100)", "lin_dec": " (h=0.01)",
                            "seas_inc": " (k=100)", "seas_dec": " (k=0.01)"}

df = pd.DataFrame()
for dataset in scores_dict.keys():
    for suffix in dataset_suffixes[dataset]:
        # create a multi index for each transformation in the dataset, with one row for original data and one row for generated data
        row_name = dataset + suffix_to_transformation[suffix]
        tuples = [(row_name, "original test set"), (row_name, ("generated test set"))]
        index = pd.MultiIndex.from_tuples(tuples)
        
        columns = []
        old_original_scores = []
        new_original_scores = []
        old_generated_scores = []
        new_generated_scores = []
        for model in scores_dict[dataset]:
            old_model_col = (model, "old")
            new_model_col =  (model, "augmented")
            columns.append(old_model_col)
            columns.append(new_model_col)
            
            old_original_scores.append(scores_dict[dataset][model][suffix]["original model"]["orig"])
            old_generated_scores.append(scores_dict[dataset][model][suffix]["original model"]["ood"])
            new_original_scores.append(scores_dict[dataset][model][suffix]["new model"]["orig"])
            new_generated_scores.append(scores_dict[dataset][model][suffix]["new model"]["ood"])
        
        
        zipped_original = []
        for old, new in zip(old_original_scores, new_original_scores):
            zipped_original.append(np.round(old, 3))
            
            percentage = np.round((np.abs(old - new) / old) * 100, 3)
            percentage_str = "+" + str(percentage) if new >= old else "-" + str(percentage)
            new = f"{np.round(new, 3)} ({percentage_str}%)"
            zipped_original.append(new)
            
            
        zipped_generated = []
        for old, new in zip(old_generated_scores, new_generated_scores):
            zipped_generated.append(np.round(old, 3))
            
            percentage = np.round((np.abs(old - new) / old) * 100, 3)
            percentage_str = "+" + str(percentage) if new >= old else "-" + str(percentage)
            new = f"{np.round(new, 3)} ({percentage_str}%)"
            zipped_generated.append(new)
        
        suffix_df = pd.DataFrame(np.vstack([zipped_original, zipped_generated]), columns=columns)
        suffix_df.index = index
        suffix_df.columns = pd.MultiIndex.from_tuples(suffix_df.columns)
        df = pd.concat([df, suffix_df])

In [11]:
df

Fully-connected  \
                                                         old   
electricity_nips (k=0.01) original test set            58.41   
                          generated test set          39.696   
electricity_nips (m=-1)   original test set            58.41   
                          generated test set         315.842   
electricity_nips (m=1)    original test set            58.41   
                          generated test set         246.497   
traffic_nips (k=0.01)     original test set            0.007   
                          generated test set           0.009   
traffic_nips (m=-1)       original test set            0.007   
                          generated test set           0.018   
traffic_nips (m=1)        original test set            0.007   
                          generated test set           0.015   
traffic_nips (f=0.01)     original test set            0.007   
                          generated test set           0.009   
m4_hourly (k=0.01)        original test set          285.752   
                          generated test set          255.06   
m4_hourly (m=-1)          original test set          285.752   
                          generated test set        4044.765   
m4_hourly (m=1)           original test set          285.752   
                          generated test set        3156.815   
m4_daily (k=100)          original test set          180.823   
                          generated test set        5097.152   
m4_weekly (m=-1)          original test set          305.795   
                          generated test set       18854.778   
m4_weekly (m=1)           original test set          305.795   
                          generated test set       26996.227   
m4_weekly (f=0.01)        original test set          305.795   
                          generated test set        1370.924   
m4_monthly (k=0.01)       original test set          599.508   
                          generated test set         516.521   
m4_monthly (m=-1)         original test set          599.508   
                          generated test set        1227.818   
m4_monthly (m=1)          original test set          599.508   
                          generated test set          730.01   
m4_monthly (h=0.01)       original test set          599.508   
                          generated test set       28597.189   
m4_monthly (f=0.01)       original test set          599.508   
                          generated test set         379.357   
m4_quarterly (k=0.01)     original test set          601.418   
                          generated test set         550.054   
m4_quarterly (k=100)      original test set          601.418   
                          generated test set         700.863   
m4_quarterly (h=0.01)     original test set          601.418   
                          generated test set       37976.754   
m4_quarterly (m=-1)       original test set          601.418   
                          generated test set        1365.815   
m4_yearly (h=0.01)        original test set          856.154   
                          generated test set       24146.394   
m4_yearly (m=-1)          original test set          856.154   
                          generated test set        2943.082   
m4_yearly (m=1)           original test set          856.154   
                          generated test set       13849.622   

                                                                         LSTM  \
                                                         augmented        old   
electricity_nips (k=0.01) original test set       63.663 (+8.994%)     53.361   
                          generated test set     30.907 (-22.141%)     43.864   
electricity_nips (m=-1)   original test set      48.889 (-16.299%)     53.361   
                          generated test set     52.343 (-83.427%)    309.964   
electricity_nips (m=1)    original test set      49.359 (-15.495%)     53.361   
                     

In [12]:
print(df.to_latex(multirow=True, column_format="|ll|cc|cc|cc|cc|cc|", multicolumn_format="c"))

\begin{tabular}{|ll|cc|cc|cc|cc|cc|}
\toprule
                &                    & \multicolumn{2}{c}{Fully-connected} & \multicolumn{2}{c}{LSTM} & \multicolumn{2}{c}{N-BEATS} & \multicolumn{2}{c}{TCN} & \multicolumn{2}{c}{Transformer} \\
                &                    &             old &             augmented &        old &             augmented &         old &             augmented &        old &             augmented &         old &             augmented \\
\midrule
\multirow{2}{*}{electricity\_nips (k=0.01)} & original test set &           58.41 &      63.663 (+8.994\%) &     53.361 &      59.56 (+11.618\%) &      44.002 &      44.284 (+0.642\%) &     53.067 &     74.053 (+39.547\%) &      51.421 &      53.983 (+4.982\%) \\
                & generated test set &          39.696 &     30.907 (-22.141\%) &     43.864 &      42.126 (-3.962\%) &      38.066 &     25.911 (-31.933\%) &      39.43 &      35.993 (-8.715\%) &      40.159 &     32.635 (-18.736\%) \\
\cline{1-12}
\mul